In [1]:
import pandas as pd

from src.metrics import map_score, mrr_score, ndcg_score, rmse_score
from src.models.alternating_least_squares import ALSRecommender
from src.utils import train_test_split, to_user_movie_matrix, make_binary_matrix, RatingMatrix

Let's load the datasets with users info, movies info and users' ratings for movies.

Then we split it to training/test subsets by the timestamp.

In [2]:
users = pd.read_table("../data/users.dat", sep="::", names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], engine='python')

movies = pd.read_table("../data/movies.dat", sep="::", names=['MovieID', 'Title', 'Genres'], engine='python', encoding='latin1')

ratings = pd.read_table("../data/ratings.dat", sep="::", names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')
ratings['Timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')

ratings = ratings[ratings['MovieID'].isin(movies['MovieID'])]

train_ratings, test_ratings = train_test_split(ratings, 'Timestamp')
user_movie_train = to_user_movie_matrix(train_ratings)
user_movie_test = to_user_movie_matrix(test_ratings)

Now, we are going to train our ALS recommender model. It depends on two parameters factor and alpha, so we will do the cross validation to find optimal values based on 4 metrics:
* mean average precision (MAP)
* mean reciprocal rank (MRR)
* normalized discounted cumulative gain (NDCG)
* root mean squared error (RMSE)

In [3]:
import numpy as np
from tqdm import tqdm

factors = [1,2,4,8]
alphas = np.linspace(1e-12, 0.05, 20)

In [4]:
map_list = list()
mrr_list = list()
ndcg_list = list()
rmse_list = list()
for factor in tqdm(factors):
    for alpha in tqdm(alphas):
        model = ALSRecommender()
        model.train(user_movie_train, factor, lambda_=alpha)
        
        y_pred = model.predict(make_binary_matrix(user_movie_test.get_rating_matrix()))
        map_list.append({"factor": factor, "lambda": alpha, "map": map_score(RatingMatrix(user_movie_test.get_rating_matrix()[y_pred.get_users()]), y_pred, top=10)})
        mrr_list.append({"factor": factor, "lambda": alpha, "mrr": mrr_score(RatingMatrix(user_movie_test.get_rating_matrix()[y_pred.get_users()]), y_pred, top=10)})
        ndcg_list.append({"factor": factor, "lambda": alpha, "ndcg": ndcg_score(RatingMatrix(user_movie_test.get_rating_matrix()[y_pred.get_users()]), y_pred, top=10)})
        rmse_list.append({"factor": factor, "lambda": alpha, "rmse": rmse_score(RatingMatrix(user_movie_test.get_rating_matrix()[y_pred.get_users()]), y_pred)})

100%|██████████| 4/4 [16:29<00:00, 247.42s/it]


In [9]:
map_df = pd.DataFrame(map_list)
mrr_df = pd.DataFrame(mrr_list)
ndcg_df = pd.DataFrame(ndcg_list)
rmse_df = pd.DataFrame(rmse_list)

In [13]:
map_df[map_df['map'] == map_df['map'].max()]

,factor,lambda,map
65,8,0.013158,0.200768


In [14]:
mrr_df[mrr_df['mrr'] == mrr_df['mrr'].max()]

,factor,lambda,mrr
70,8,0.026316,0.488619


In [15]:
ndcg_df[ndcg_df['ndcg'] == ndcg_df['ndcg'].max()]

,factor,lambda,ndcg
77,8,0.044737,0.289001


In [16]:
rmse_df[rmse_df['rmse'] == rmse_df['rmse'].min()]

,factor,lambda,rmse
74,8,0.036842,3.240899


The best possible MAP is 0.200 indicates that, on average, about 20.0% of the top-10 recommended items are relevant.

The best possible MRR is 0.488 means that, on average, the first relevant item appears between the 2nd and 3rd positions in the recommendation list.

NDCG measures the quality of the recommendations by considering the position of the relevant items in the list, with higher-ranked items contributing more to the score. The best possible NDCG is 0.289

RMSE measures the differences between the predicted and actual ratings. The best possible RMSE is 3.24.